# Process image folder

In [ ]:
import os
import cv2
from deepface import DeepFace

folder = './faces'

print(folder)

for file_name in os.listdir(folder):
    # Asume imágenes en formato png o jpg
    if file_name.endswith('.png') or file_name.endswith('.jpg'):
        # Procesa la imagen que asume hay cara, no fuerza la detección
        obj = DeepFace.analyze(img_path = os.path.join(folder, file_name), enforce_detection=False, actions =['age', 'gender', 'race', 'emotion'])
        print(file_name)
        print(obj)
        #print(obj["region"])
        #print(obj["age"])      
        #print(obj["gender"])      
        #print(obj["race"])       
        #print(obj["dominant_race"]) 
        #print(obj["emotion"])
        #print(obj["dominant_emotion"])
       
        


# Process from webcam

In [1]:
import cv2
import numpy as np
from deepface import DeepFace
#from playsound import playsound

vid = cv2.VideoCapture(0)

# font 
font = cv2.FONT_HERSHEY_SIMPLEX
fontScale = 1
color = (255, 0, 0) 
thickness = 2 
pos = (50, 50)

def blend(input_rgb, alpha, rgb, coord: (int, int)):
    cut = input_rgb[coord[1]:coord[1]+rgb.shape[0], coord[0]:coord[0]+rgb.shape[1]].astype("float32")

    # Multiply the foreground with the alpha matte
    rgb = cv2.multiply(cv2.cvtColor(alpha, cv2.COLOR_GRAY2BGR), rgb)
    cut = cv2.multiply(cv2.cvtColor(1.0 - alpha, cv2.COLOR_GRAY2BGR), cut)

    # Add the masked foreground and background

    input_rgb[coord[1]:coord[1]+rgb.shape[0], coord[0]:coord[0]+rgb.shape[1]] = cv2.add(rgb, cut).astype("uint8")
    return input_rgb

def read_transparent_png(filename):
    image_4channel = cv2.imread(filename, cv2.IMREAD_UNCHANGED)
    alpha_channel = image_4channel[:,:,3].astype("float32") / 255
    rgb_channels = image_4channel[:,:,:3].astype("float32")
    return (alpha_channel, rgb_channels)

balloons = read_transparent_png("assets/balloons.png")
confetti = read_transparent_png("assets/confetti.png")
rain = read_transparent_png("assets/rain.png")
sangre1 = read_transparent_png("assets/sangre1.png")
serious = read_transparent_png("assets/serious.png")
vignette = read_transparent_png("assets/vignette.png")
stars = read_transparent_png("assets/stars.png")

while(True):
    # fotograma a fotograma
    ret, frame = vid.read()

    if ret:  
        obj = DeepFace.analyze(img_path = frame, enforce_detection=False, actions =['gender', 'emotion'])
        print(obj[0]['dominant_emotion'])
        # Primera cara
        #image = cv2.putText(frame, str(obj[0]['dominant_gender']+' '+obj[0]['dominant_emotion']), pos, font,  
        #                fontScale, color, thickness, cv2.LINE_AA) 

        match obj[0]['dominant_emotion']: 
            case 'happy':
                imghsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV).astype("float32")

                (h, s, v) = cv2.split(imghsv)
                s = s*2
                s = np.clip(s,0,255)
                imghsv = cv2.merge([h,s,v])

                frame = cv2.cvtColor(imghsv.astype("uint8"), cv2.COLOR_HSV2BGR)
                frame = blend(frame, balloons[0], balloons[1], (0, 0))
                frame = blend(frame, stars[0], stars[1], (0, 0))

            case 'sad':
                frame = blend(frame, rain[0], rain[1], (0, 0))
                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

            case 'angry':
                frame2 = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                image = np.zeros((frame.shape), dtype = np.uint8)
                image[:,:,2] = frame2
                frame = image

            case 'fear':
                imghsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV).astype("float32")

                (h, s, v) = cv2.split(imghsv)
                s = 0.5*s
                s = np.clip(s,0,255)
                imghsv = cv2.merge([h,s,v])
                frame = cv2.cvtColor(imghsv.astype("uint8"), cv2.COLOR_HSV2BGR)
                frame = blend(frame, vignette[0], vignette[1], (0, 0))

            case 'disgust':
                frame2 = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                image = np.zeros((frame.shape), dtype = np.uint8)
                image[:,:,1] = frame2
                frame = image

            case 'surprise':
                frame = blend(frame, confetti[0], confetti[1], (0, 0))
                frame = blend(frame, stars[0], stars[1], (0, 0))

            case 'neutral':
                frame = blend(frame, serious[0], serious[1], (0, 0))

            case _:
                pass

        # Muestra fotograma
        cv2.imshow('Vid', frame)

    # Detenemos pulsado ESC
    if cv2.waitKey(20) == 27:
        break

# Libera el objeto de captura
vid.release()
# Destruye ventanas
cv2.destroyAllWindows()

2023-12-07 11:46:03.829705: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-07 11:46:04.183734: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-07 11:46:04.183804: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-07 11:46:04.242912: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-07 11:46:04.366139: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-07 11:46:04.367509: I tensorflow/core/platform/cpu_feature_guard.cc:1

angry


Action: emotion: 100%|██████████| 2/2 [00:00<00:00,  6.63it/s]


angry


Action: emotion: 100%|██████████| 2/2 [00:00<00:00,  6.76it/s]


...

happy


Action: emotion: 100%|██████████| 2/2 [00:00<00:00,  6.39it/s]


happy


Action: emotion: 100%|██████████| 2/2 [00:00<00:00,  5.67it/s]

happy
